In [3]:
from transformers import pipeline
from transformers.image_utils import load_image

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg"
image = load_image(url)

feature_extractor = pipeline(
    model="facebook/dinov3-convnext-tiny-pretrain-lvd1689m",
    task="image-feature-extraction", 
)
features = feature_extractor(image)

ConnectTimeout: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7df4f5739b50>, 'Connection to huggingface.co timed out. (connect timeout=None)'))

In [4]:
import os
import cv2
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from skimage.segmentation import felzenszwalb
import pickle
import warnings
warnings.filterwarnings('ignore')

class SizeAwareSeismicDetector:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.model = self._load_dinov3_model()
        
        # DINOv3 的 patch 大小
        self.patch_size = 800
        
        # 图像尺寸统计
        self.avg_width = 0
        self.avg_height = 0
        self.min_width = float('inf')
        self.min_height = float('inf')
        self.max_width = 0
        self.max_height = 0
        
        # 参考特征库
        self.positive_features = []
        self.negative_features = []
        self.positive_clusters = None
        self.negative_clusters = None
        
        # 检测参数（将根据图像尺寸动态调整）
        self.positive_threshold = 0.75
        self.negative_threshold = 0.65
        self.min_region_size = 100
        
    def _load_dinov3_model(self):
        """加载DINOv3模型"""
        try:
            model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb16')
            model.eval()
            return model.to(self.device)
        except:
            from transformers import AutoImageProcessor, AutoModel
            model = AutoModel.from_pretrained('facebook/dinov2-base')
            model.eval()
            return model.to(self.device)
    
    def _make_size_divisible(self, size, divisor=800):
        """
        调整尺寸使其可被指定数值整除
        Args:
            size: 原始尺寸 (width, height) 或单个尺寸值
            divisor: 除数 (DINOv3的patch_size是14)
        Returns:
            调整后的尺寸
        """
        if isinstance(size, (tuple, list)):
            w, h = size
            new_w = (w // divisor) * divisor
            new_h = (h // divisor) * divisor
            return (new_w, new_h)
        else:
            return (size // divisor) * divisor
    
    def analyze_image_sizes(self, directory):
        """
        分析目录中图像的尺寸分布，并确保尺寸可被patch_size整除
        Args:
            directory: 图像目录
        Returns:
            size_stats: 尺寸统计信息
        """
        widths = []
        heights = []
        
        if not os.path.exists(directory):
            print(f"警告: {directory} 目录不存在")
            return None
        
        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']:
            image_files.extend([os.path.join(directory, f) for f in os.listdir(directory) 
                              if f.lower().endswith(ext[1:])])
        
        if not image_files:
            print(f"警告: {directory} 目录中没有找到图片文件")
            return None
        
        print(f"分析 {len(image_files)} 张图像的尺寸...")
        
        for img_path in image_files:
            try:
                with Image.open(img_path) as img:
                    width, height = img.size
                    
                    # 调整尺寸使其可被patch_size整除
                    adj_width, adj_height = self._make_size_divisible((width, height), self.patch_size)
                    
                    widths.append(adj_width)
                    heights.append(adj_height)
                    
                    # 更新统计信息
                    self.min_width = min(self.min_width, adj_width)
                    self.min_height = min(self.min_height, adj_height)
                    self.max_width = max(self.max_width, adj_width)
                    self.max_height = max(self.max_height, adj_height)
                    
            except Exception as e:
                print(f"分析图像 {img_path} 尺寸时出错: {e}")
        
        if widths and heights:
            self.avg_width = np.mean(widths)
            self.avg_height = np.mean(heights)
            
            size_stats = {
                'avg_width': self.avg_width,
                'avg_height': self.avg_height,
                'min_width': self.min_width,
                'min_height': self.min_height,
                'max_width': self.max_width,
                'max_height': self.max_height,
                'total_images': len(image_files)
            }
            
            print(f"图像尺寸分析完成:")
            print(f"  平均尺寸: {self.avg_width:.1f} x {self.avg_height:.1f}")
            print(f"  最小尺寸: {self.min_width} x {self.min_height}")
            print(f"  最大尺寸: {self.max_width} x {self.max_height}")
            print(f"  总图像数: {len(image_files)}")
            
            return size_stats
        else:
            return None
    
    def get_adaptive_transform(self, target_size=None):
        """
        根据图像尺寸获取自适应变换，确保尺寸可被patch_size整除
        Args:
            target_size: 目标尺寸，如果为None则使用平均尺寸
        Returns:
            transform: 图像变换
        """
        if target_size is None:
            # 使用平均尺寸，但限制在合理范围内且可被patch_size整除
            target_w = min(800, int(self.avg_width))
            target_h = min(800, int(self.avg_height))
            target_w, target_h = self._make_size_divisible((target_w, target_h), self.patch_size)
            target_size = (target_h, target_w)
        else:
            # 确保目标尺寸可被patch_size整除
            if isinstance(target_size, int):
                target_size = self._make_size_divisible(target_size, self.patch_size)
                target_size = (target_size, target_size)
            else:
                target_h, target_w = target_size
                target_h = self._make_size_divisible(target_h, self.patch_size)
                target_w = self._make_size_divisible(target_w, self.patch_size)
                target_size = (target_h, target_w)
        
        print(f"使用图像尺寸: {target_size[1]} x {target_size[0]} (可被{self.patch_size}整除)")
        
        return T.Compose([
            T.Resize(target_size),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225]),
        ])
    
    def extract_single_feature(self, image, transform=None):
        """提取单张图像特征，确保尺寸符合要求"""
        if isinstance(image, str):
            image = Image.open(image).convert('RGB')
        
        if transform is None:
            transform = self.get_adaptive_transform()
        
        # 确保图像尺寸可被patch_size整除
        width, height = image.size
        if width % self.patch_size != 0 or height % self.patch_size != 0:
            new_width = (width // self.patch_size) * self.patch_size
            new_height = (height // self.patch_size) * self.patch_size
            image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
            print(f"调整图像尺寸从 {width}x{height} 到 {new_width}x{new_height}")
        
        with torch.no_grad():
            inputs = transform(image).unsqueeze(0).to(self.device)
            features = self.model(inputs)
            features = torch.nn.functional.normalize(features, dim=1)
            return features.cpu().numpy()[0]
    
    def build_reference_library(self, positive_dir, negative_dir, max_samples_per_class=1000):
        """
        构建参考特征库，基于实际图像尺寸
        Args:
            positive_dir: 正类图片目录
            negative_dir: 负类图片目录
            max_samples_per_class: 每类最大样本数
        """
        print("开始构建参考特征库...")
        
        # 分析正类和负类图像的尺寸
        print("分析正类图像尺寸...")
        pos_stats = self.analyze_image_sizes(positive_dir)
        print("分析负类图像尺寸...")
        neg_stats = self.analyze_image_sizes(negative_dir)
        
        # 使用平均尺寸创建变换，确保可被patch_size整除
        self.reference_transform = self.get_adaptive_transform()
        
        # 加载正类图片特征
        self.positive_features = self._load_directory_features(
            positive_dir, max_samples_per_class, "正类", self.reference_transform)
        
        # 加载负类图片特征
        self.negative_features = self._load_directory_features(
            negative_dir, max_samples_per_class, "负类", self.reference_transform)
        
        # 对特征进行聚类，减少检索复杂度
        self._cluster_features()
        
        print(f"参考特征库构建完成: 正类 {len(self.positive_features)} 个样本, "
              f"负类 {len(self.negative_features)} 个样本")
        
        # 根据图像尺寸调整检测参数
        self._adjust_detection_parameters()
    
    def _load_directory_features(self, directory, max_samples, class_name, transform):
        """加载目录中的图片并提取特征"""
        features = []
        
        if not os.path.exists(directory):
            print(f"警告: {directory} 目录不存在")
            return features
        
        image_files = []
        for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']:
            image_files.extend([os.path.join(directory, f) for f in os.listdir(directory) 
                              if f.lower().endswith(ext[1:])])
        
        if not image_files:
            print(f"警告: {directory} 目录中没有找到图片文件")
            return features
        
        print(f"正在处理{class_name}图片: {len(image_files)} 张")
        
        # 限制样本数量
        image_files = image_files[:max_samples]
        
        for i, img_path in enumerate(image_files):
            try:
                feature = self.extract_single_feature(img_path, transform)
                features.append(feature)
                
                if (i + 1) % 50 == 0:
                    print(f"已处理 {class_name} 图片: {i + 1}/{len(image_files)}")
                    
            except Exception as e:
                print(f"处理图片 {img_path} 时出错: {e}")
                # 尝试使用默认尺寸处理
                try:
                    print("尝试使用默认尺寸处理...")
                    default_transform = self.get_adaptive_transform((224, 224))
                    feature = self.extract_single_feature(img_path, default_transform)
                    features.append(feature)
                except Exception as e2:
                    print(f"使用默认尺寸处理也失败: {e2}")
        
        return np.array(features)
    
    def _cluster_features(self, n_clusters=50):
        """对特征进行聚类，减少检索复杂度"""
        if len(self.positive_features) > 0 and len(self.positive_features) > n_clusters:
            kmeans_positive = KMeans(n_clusters=n_clusters, random_state=42)
            kmeans_positive.fit(self.positive_features)
            self.positive_clusters = kmeans_positive.cluster_centers_
            print(f"正类特征聚类为 {n_clusters} 个中心")
        elif len(self.positive_features) > 0:
            self.positive_clusters = self.positive_features
            print(f"正类特征数量较少，使用所有 {len(self.positive_features)} 个特征")
        else:
            self.positive_clusters = np.array([])
            print("警告: 没有正类特征可用")
        
        if len(self.negative_features) > 0 and len(self.negative_features) > n_clusters:
            kmeans_negative = KMeans(n_clusters=n_clusters, random_state=42)
            kmeans_negative.fit(self.negative_features)
            self.negative_clusters = kmeans_negative.cluster_centers_
            print(f"负类特征聚类为 {n_clusters} 个中心")
        elif len(self.negative_features) > 0:
            self.negative_clusters = self.negative_features
            print(f"负类特征数量较少，使用所有 {len(self.negative_features)} 个特征")
        else:
            self.negative_clusters = np.array([])
            print("警告: 没有负类特征可用")
    
    def _adjust_detection_parameters(self):
        """根据图像尺寸调整检测参数"""
        # 根据图像平均尺寸调整最小区域大小
        avg_area = self.avg_width * self.avg_height
        if avg_area > 1000000:  # 大图像
            self.min_region_size = 500
        elif avg_area > 500000:  # 中等图像
            self.min_region_size = 200
        else:  # 小图像
            self.min_region_size = 50
            
        print(f"根据图像尺寸调整参数: 最小区域大小 = {self.min_region_size}")
    
    def save_reference_library(self, save_path):
        """保存参考特征库"""
        data = {
            'positive_features': self.positive_features,
            'negative_features': self.negative_features,
            'positive_clusters': self.positive_clusters,
            'negative_clusters': self.negative_clusters,
            'avg_width': self.avg_width,
            'avg_height': self.avg_height,
            'min_width': self.min_width,
            'min_height': self.min_height,
            'max_width': self.max_width,
            'max_height': self.max_height
        }
        
        with open(save_path, 'wb') as f:
            pickle.dump(data, f)
        
        print(f"参考特征库已保存到: {save_path}")
    
    def load_reference_library(self, load_path):
        """加载参考特征库"""
        with open(load_path, 'rb') as f:
            data = pickle.load(f)
        
        self.positive_features = data['positive_features']
        self.negative_features = data['negative_features']
        self.positive_clusters = data['positive_clusters']
        self.negative_clusters = data['negative_clusters']
        self.avg_width = data['avg_width']
        self.avg_height = data['avg_height']
        self.min_width = data['min_width']
        self.min_height = data['min_height']
        self.max_width = data['max_width']
        self.max_height = data['max_height']
        
        # 根据加载的尺寸信息调整参数
        self._adjust_detection_parameters()
        
        # 使用平均尺寸创建变换
        self.reference_transform = self.get_adaptive_transform()
        
        print(f"参考特征库已加载: 正类 {len(self.positive_features)} 个样本, "
              f"负类 {len(self.negative_features)} 个样本")
        print(f"图像尺寸: 平均 {self.avg_width:.1f}x{self.avg_height:.1f}")
    
    def calculate_similarity_scores(self, query_features):
        """
        计算查询特征与参考库的相似度得分
        Returns:
            positive_scores: 与正类的相似度
            negative_scores: 与负类的相似度
            final_scores: 最终得分 (正类相似度 - 负类相似度)
        """
        if len(query_features.shape) == 1:
            query_features = query_features.reshape(1, -1)
        
        # 检查是否有参考特征
        if len(self.positive_clusters) == 0 or len(self.negative_clusters) == 0:
            print("警告: 参考特征库不完整，无法计算相似度")
            return np.zeros(len(query_features)), np.zeros(len(query_features)), np.zeros(len(query_features))
        
        # 计算与正类聚类中心的相似度
        positive_similarities = cosine_similarity(query_features, self.positive_clusters)
        positive_scores = np.max(positive_similarities, axis=1)
        
        # 计算与负类聚类中心的相似度
        negative_similarities = cosine_similarity(query_features, self.negative_clusters)
        negative_scores = np.max(negative_similarities, axis=1)
        
        # 最终得分 = 正类相似度 - 负类相似度
        final_scores = positive_scores - negative_scores
        
        return positive_scores, negative_scores, final_scores
    
    def size_aware_crop(self, image):
        """
        基于图像实际尺寸的自适应裁剪，确保尺寸可被patch_size整除
        Args:
            image: 输入图像 (PIL Image)
        Returns:
            crops: 裁剪区域列表
            positions: 对应位置信息
        """
        crops = []
        positions = []
        
        img_array = np.array(image)
        h, w = img_array.shape[:2]
        
        # 根据图像尺寸确定裁剪策略
        if w > 1000 and h > 1000:
            # 大图像: 使用较小的裁剪尺寸和较大的步长
            crop_size = min(400, min(w, h) // 3)
            stride = crop_size // 2
            scales = [0.8, 1.0, 1.2]
        elif w > 500 and h > 500:
            # 中等图像: 中等裁剪尺寸
            crop_size = min(300, min(w, h) // 2)
            stride = crop_size // 2
            scales = [0.9, 1.0, 1.1]
        else:
            # 小图像: 使用较小的裁剪尺寸
            crop_size = min(224, min(w, h))
            stride = crop_size // 2
            scales = [1.0]
        
        for scale in scales:
            current_crop_size = int(crop_size * scale)
            # 确保裁剪尺寸可被patch_size整除
            current_crop_size = self._make_size_divisible(current_crop_size, self.patch_size)
            # 确保裁剪尺寸不超过图像尺寸
            current_crop_size = min(current_crop_size, min(w, h))
            
            for y in range(0, h - current_crop_size + 1, stride):
                for x in range(0, w - current_crop_size + 1, stride):
                    crop = img_array[y:y+current_crop_size, x:x+current_crop_size]
                    
                    # 使用参考库的变换尺寸调整裁剪区域
                    crop_pil = Image.fromarray(crop)
                    crop_resized = self.reference_transform(crop_pil).unsqueeze(0)
                    
                    crops.append(crop_resized)
                    positions.append((x, y, x+current_crop_size, y+current_crop_size, scale))
        
        return crops, positions
    
    def selective_search(self, image, scale=None, sigma=0.9, min_size=None):
        """
        基于图像尺寸的选择性搜索，确保区域尺寸可被patch_size整除
        Args:
            image: 输入图像
            scale: 分割尺度，如果为None则根据图像尺寸自动确定
            min_size: 最小区域尺寸，如果为None则根据图像尺寸自动确定
        Returns:
            regions: 候选区域列表
        """
        img_array = np.array(image)
        h, w = img_array.shape[:2]
        
        # 根据图像尺寸自动调整参数
        if scale is None:
            # 图像面积越大，scale值越小（更精细的分割）
            area = w * h
            if area > 1000000:
                scale = 300
            elif area > 500000:
                scale = 500
            else:
                scale = 800
        
        if min_size is None:
            min_size = self.min_region_size
        
        segments = felzenszwalb(img_array, scale=scale, sigma=sigma, min_size=min_size)
        
        regions = []
        unique_segments = np.unique(segments)
        
        for seg_val in unique_segments:
            mask = segments == seg_val
            if np.sum(mask) < min_size:
                continue
                
            coords = np.column_stack(np.where(mask))
            y_min, x_min = coords.min(axis=0)
            y_max, x_max = coords.max(axis=0)
            
            # 根据图像尺寸调整边界框扩展
            margin = max(3, min(w, h) // 200)
            x_min = max(0, x_min - margin)
            y_min = max(0, y_min - margin)
            x_max = min(w, x_max + margin)
            y_max = min(h, y_max + margin)
            
            # 确保区域尺寸可被patch_size整除
            region_width = x_max - x_min
            region_height = y_max - y_min
            if region_width % self.patch_size != 0 or region_height % self.patch_size != 0:
                # 调整区域尺寸
                new_width = (region_width // self.patch_size) * self.patch_size
                new_height = (region_height // self.patch_size) * self.patch_size
                if new_width > 0 and new_height > 0:
                    x_max = x_min + new_width
                    y_max = y_min + new_height
                    # 确保不超出图像边界
                    if x_max > w:
                        x_max = w
                        x_min = max(0, x_max - new_width)
                    if y_max > h:
                        y_max = h
                        y_min = max(0, y_max - new_height)
            
            region = img_array[y_min:y_max, x_min:x_max]
            if region.size == 0:
                continue
                
            region_pil = Image.fromarray(region)
            # 使用参考库的变换处理区域
            region_tensor = self.reference_transform(region_pil).unsqueeze(0)
            
            regions.append({
                'region': region_tensor,
                'bbox': (x_min, y_min, x_max, y_max),
                'size': (x_max - x_min) * (y_max - y_min)
            })
        
        return regions
    
    def extract_batch_features(self, tensor_list):
        """批量提取特征"""
        if not tensor_list:
            return np.array([])
            
        with torch.no_grad():
            # 假设tensor_list已经是处理好的tensor
            if len(tensor_list) == 1:
                inputs = tensor_list[0].to(self.device)
            else:
                inputs = torch.cat(tensor_list, dim=0).to(self.device)
            
            features = self.model(inputs)
            features = torch.nn.functional.normalize(features, dim=1)
            return features.cpu().numpy()
    
    def detect_events(self, image_path, confidence_threshold=0.1, visualize=True):
        """
        基于参考特征库和图像尺寸的地震事件检测
        Args:
            image_path: 待检测图像路径
            confidence_threshold: 置信度阈值
            visualize: 是否可视化结果
        """
        print("开始地震事件检测...")
        
        # 加载图像
        image = Image.open(image_path).convert('RGB')
        img_width, img_height = image.size
        print(f"检测图像尺寸: {img_width} x {img_height}")
        
        # 确保图像尺寸可被patch_size整除
        if img_width % self.patch_size != 0 or img_height % self.patch_size != 0:
            new_width = (img_width // self.patch_size) * self.patch_size
            new_height = (img_height // self.patch_size) * self.patch_size
            image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
            print(f"调整图像尺寸从 {img_width}x{img_height} 到 {new_width}x{new_height}")
        
        # 步骤1: 基于尺寸的自适应裁剪
        print("步骤1: 自适应裁剪...")
        crops, crop_positions = self.size_aware_crop(image)
        print(f"生成 {len(crops)} 个裁剪区域")
        
        # 步骤2: 选择性搜索
        print("步骤2: 选择性搜索...")
        regions = self.selective_search(image)
        print(f"生成 {len(regions)} 个候选区域")
        
        # 步骤3: 提取特征
        print("步骤3: 提取特征...")
        all_regions = crops + [r['region'] for r in regions]
        all_positions = crop_positions + [r['bbox'] for r in regions]
        
        if len(all_regions) == 0:
            print("未生成任何候选区域")
            return []
        
        region_features = self.extract_batch_features(all_regions)
        
        # 步骤4: 基于参考库的相似度计算
        print("步骤4: 相似度检索...")
        positive_scores, negative_scores, final_scores = self.calculate_similarity_scores(region_features)
        
        # 步骤5: 生成检测结果
        print("步骤5: 生成检测结果...")
        detections = []
        for i, (pos_score, neg_score, final_score) in enumerate(zip(positive_scores, negative_scores, final_scores)):
            # 筛选条件: 正类相似度高且最终置信度达标
            if pos_score > self.positive_threshold and final_score > confidence_threshold:
                bbox = all_positions[i]
                if len(bbox) == 5:  # 裁剪区域位置信息
                    bbox = bbox[:4]  # 取前四个坐标
                
                detections.append({
                    'bbox': bbox,
                    'confidence': final_score,
                    'positive_similarity': pos_score,
                    'negative_similarity': neg_score,
                    'feature': region_features[i]
                })
        
        # 步骤6: 后处理
        print("步骤6: 后处理筛选...")
        final_detections = self.non_maximum_suppression(detections, iou_threshold=0.3)
        
        print(f"检测完成! 发现 {len(final_detections)} 个潜在地震事件")
        
        if visualize:
            self.visualize_results(image, final_detections)
        
        return final_detections
    
    def non_maximum_suppression(self, detections, iou_threshold=0.5):
        """非极大值抑制"""
        if not detections:
            return []
        
        def calculate_iou(box1, box2):
            x1 = max(box1[0], box2[0])
            y1 = max(box1[1], box2[1])
            x2 = min(box1[2], box2[2])
            y2 = min(box1[3], box2[3])
            
            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
            area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
            union = area1 + area2 - intersection
            
            return intersection / union if union > 0 else 0
        
        detections.sort(key=lambda x: x['confidence'], reverse=True)
        
        filtered_detections = []
        while detections:
            best = detections.pop(0)
            filtered_detections.append(best)
            
            detections = [det for det in detections 
                         if calculate_iou(best['bbox'], det['bbox']) < iou_threshold]
        
        return filtered_detections
    
    def visualize_results(self, image, detections):
        """可视化检测结果"""
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        ax.imshow(image)
        
        for i, det in enumerate(detections):
            bbox = det['bbox']
            confidence = det['confidence']
            
            rect = patches.Rectangle((bbox[0], bbox[1]), 
                                   bbox[2]-bbox[0], bbox[3]-bbox[1],
                                   linewidth=2, edgecolor='red', 
                                   facecolor='none')
            ax.add_patch(rect)
            
            ax.text(bbox[0], bbox[1]-5, f'Conf: {confidence:.3f}', 
                   color='red', fontsize=10, weight='bold')
        
        ax.set_title(f'Seismic Event Detection - Found {len(detections)} events')
        ax.axis('off')
        plt.tight_layout()
        plt.show()

In [5]:

# 使用示例
def main():
    # 初始化检测器
    detector = SizeAwareSeismicDetector()
    
    # 定义正负类图片目录
    positive_dir = "/home/disk/wd_black/dinov3/data/positive"  # 包含地震事件的图片
    negative_dir = "/home/disk/wd_black/dinov3/data/negative"  # 不包含地震事件的图片
    
    # 构建或加载参考特征库
    reference_lib_path = "seismic_reference_library.pkl"
    
    # if os.path.exists(reference_lib_path):
    #     print("加载现有参考特征库...")
    #     detector.load_reference_library(reference_lib_path)
    # else:
    print("构建新的参考特征库...")
    detector.build_reference_library(positive_dir, negative_dir)
    detector.save_reference_library(reference_lib_path)
    
    # 检测地震事件
    test_image_path = "/home/disk/wd_black/dinov3/data/test_image/anyuan_cut_213_20230808_063302.379000.png"
    
    try:
        events = detector.detect_events(test_image_path, confidence_threshold=0.1, visualize=True)
        
        # 输出检测结果
        print("\n检测结果汇总:")
        for i, event in enumerate(events):
            print(f"事件 {i+1}: 位置 {event['bbox']}, "
                  f"置信度 {event['confidence']:.3f}, "
                  f"正类相似度 {event['positive_similarity']:.3f}, "
                  f"负类相似度 {event['negative_similarity']:.3f}")
            
    except Exception as e:
        print(f"检测过程中出现错误: {e}")

if __name__ == "__main__":
    main()

Using cache found in /home/wzm/.cache/torch/hub/facebookresearch_dinov2_main
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /facebook/dinov2-base/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7df4f5fa5f50>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 1ee1e5b8-466c-40eb-93e8-a9961c07acfc)')' thrown while requesting HEAD https://huggingface.co/facebook/dinov2-base/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /facebook/dinov2-base/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7df4f5f99890>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 513d8277-a4a5-4ac4-973e-c7bfd7615de6)')' thrown while requesting HEAD https://huggingface.co/facebook/dino

OSError: We couldn't connect to 'https://huggingface.co' to load the files, and couldn't find them in the cached files.
Check your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.